<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/scrapingWikiList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import urllib.request
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
print('imports successful')
starturl = 'https://en.wikipedia.org'

imports successful


In [48]:
import pprint
def bio_table(url):
    page = urllib.request.urlopen(url)
    soup = bs(page, "lxml")
    table = soup.find('table', class_='infobox biography vcard')
    print(len(table.find_all('ul', class_='NavContent')))
    influencers = table.find_all('ul', class_='NavContent')[0]
    influenced = table.find_all('ul', class_='NavContent')[1]
    print(influenced)
    final_influencers = []
    final_influenced = []
    for a in influencers.find_all('a'):
        try:
            final_influencers.append(a.get('title'))
        except:
            pass
    for a in influenced.find_all('a'):
        try:
            final_influenced.append(a.get('title'))
        except:
            pass
    '''
    exceptional_row_count = 0
    for tr in table.find_all('tr'):
        if tr.find('th'):
            result[tr.find('th').text] = tr.find('td').text
        else:
            # the first row Logos fall here
            exceptional_row_count += 1
    if exceptional_row_count > 1:
        print ('WARNING ExceptionalRow>1: ' + table)
    '''
    return final_influencers,final_influenced


pprint.pprint(bio_table('https://en.wikipedia.org/wiki/Anaximander'))

2
<ul class="NavContent" style="list-style: none none; margin-left: 0; text-align: left; font-size: 105%; margin-top: 0; margin-bottom: 0; line-height: inherit;"><li style="line-height: inherit; margin: 0"><div class="center" style="width:auto; margin-left:auto; margin-right:auto;">All pre-socratic philosophy, in particular: <a href="/wiki/Anaximenes_of_Miletus" title="Anaximenes of Miletus">Anaximenes</a>, <a href="/wiki/Pythagoras" title="Pythagoras">Pythagoras</a>, <a href="/wiki/Democritus" title="Democritus">Democritus</a>, <a class="mw-redirect" href="/wiki/Greek_astronomy" title="Greek astronomy">Greek astronomy</a>; <a href="/wiki/Martin_Heidegger" title="Martin Heidegger">Heidegger</a></div></li></ul>
(['Thales'],
 ['Anaximenes of Miletus',
  'Pythagoras',
  'Democritus',
  'Greek astronomy',
  'Martin Heidegger'])


Philsophers from A to C

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_philosophers_(A%E2%80%93C)"
page = urllib.request.urlopen(url)
soup = bs(page, "lxml")
#print(soup.prettify())
#print(soup.title.string)

In [51]:

phil=[]
href=[]
raw=[]
influencers=[]
influenced=[]
for ultag in soup.find_all('ul'):
  for litag in ultag.find_all('li'):
    try:
      print('HREF: ' + litag.a.get('href'))
      wikiurl = starturl + litag.a.get('href')
      print(wikiurl)
      href.append(wikiurl)
    except:
      print('No HREF')
      href.append('NONE')
    try:
        response = requests.get(
            'https://en.wikipedia.org/w/api.php',
            params={
                'action': 'query',
                'format': 'json',
                'titles': litag.a.get('title'),
                'prop': 'extracts',
                'explaintext': True,
                'exlimit': 'max',
            }
        ).json()
        raw.append(response)
    except:
        raw.append('NONE')
    try:
        tempinfluencers, tempinfluenced = bio_table(wikiurl)
        influencers.append(tempinfluencers)
        influenced.append(tempinfluenced)
    except:
        influencers.append([])
        influenced.append([])
    phil.append(litag.text)
    print(litag.text)




HREF: /wiki/Philosophy
https://en.wikipedia.org/wiki/Philosophy
Article
HREF: /wiki/Category:Philosophy
https://en.wikipedia.org/wiki/Category:Philosophy
Category
HREF: /wiki/Glossary_of_philosophy
https://en.wikipedia.org/wiki/Glossary_of_philosophy
Glossary
HREF: /wiki/Outline_of_philosophy
https://en.wikipedia.org/wiki/Outline_of_philosophy
Outline
HREF: /wiki/Portal:Philosophy
https://en.wikipedia.org/wiki/Portal:Philosophy
Portal
HREF: /wiki/Adi_Shankara
https://en.wikipedia.org/wiki/Adi_Shankara
0
Adi Shankara, circa. 7th Century
HREF: /wiki/Nicola_Abbagnano
https://en.wikipedia.org/wiki/Nicola_Abbagnano
0
Nicola Abbagnano, (1901–1990)[2]
HREF: /wiki/Muhammad_Abduh
https://en.wikipedia.org/wiki/Muhammad_Abduh
2
<ul class="NavContent" style="list-style: none none; margin-left: 0; text-align:center; font-size: 105%; margin-top: 0; margin-bottom: 0; line-height: inherit;"><li style="line-height: inherit; margin: 0"><a href="/wiki/Rashid_Rida" title="Rashid Rida">Rashid Rida</a>, <a 

Philosopers from D to H

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_philosophers_(D%E2%80%93H)"
page = urllib.request.urlopen(url)
soup = bs(page, "lxml")
#print(soup.prettify())
#print(soup.title.string)

In [0]:
for ultag in soup.find_all('ul'):
  for litag in ultag.find_all('li'):
    try:
      print('HREF: ' + litag.a.get('href'))
      wikiurl = starturl + litag.a.get('href')
      print(wikiurl)
      href.append(wikiurl)
    except:
      print('No HREF')
      href.append('NONE')
    try:
        response = requests.get(
            'https://en.wikipedia.org/w/api.php',
            params={
                'action': 'query',
                'format': 'json',
                'titles': litag.a.get('title'),
                'prop': 'extracts',
                'explaintext': True,
                'exlimit': 'max',
            }
        ).json()
        raw.append(response)
    except:
        raw.append('NONE')
    try:
        tempinfluencers, tempinfluenced = bio_table(wikiurl)
        influencers.append(tempinfluencers)
        influenced.append(tempinfluenced)
    except:
        influencers.append([])
        influenced.append([])
    phil.append(litag.text)
    print(litag.text)

HREF: /wiki/Philosophy
https://en.wikipedia.org/wiki/Philosophy
Article
HREF: /wiki/Category:Philosophy
https://en.wikipedia.org/wiki/Category:Philosophy
Category
HREF: /wiki/Glossary_of_philosophy
https://en.wikipedia.org/wiki/Glossary_of_philosophy
Glossary
HREF: /wiki/Outline_of_philosophy
https://en.wikipedia.org/wiki/Outline_of_philosophy
Outline
HREF: /wiki/Portal:Philosophy
https://en.wikipedia.org/wiki/Portal:Philosophy
Portal
HREF: /wiki/Jean_le_Rond_d%27Alembert
https://en.wikipedia.org/wiki/Jean_le_Rond_d%27Alembert
0
Jean le Rond d'Alembert, (1717–1783)[1][2][3][4]
HREF: /wiki/Damascius
https://en.wikipedia.org/wiki/Damascius
Damascius, (c. 462–540)[1][2][4]
HREF: /wiki/Peter_Damian
https://en.wikipedia.org/wiki/Peter_Damian
Peter Damian, (c. 1007–1072)[2][4][5]
HREF: /wiki/Hubert_Damisch
https://en.wikipedia.org/wiki/Hubert_Damisch
Hubert Damisch, (1928–2017)
HREF: /wiki/Arthur_Danto
https://en.wikipedia.org/wiki/Arthur_Danto
2
<ul class="NavContent" style="list-style: non

Philosophers from I to Q

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_philosophers_(I%E2%80%93Q)"
page = urllib.request.urlopen(url)
soup = bs(page, "lxml")

In [0]:
for ultag in soup.find_all('ul'):
  for litag in ultag.find_all('li'):
    try:
      print('HREF: ' + litag.a.get('href'))
      wikiurl = starturl + litag.a.get('href')
      print(wikiurl)
      href.append(wikiurl)
    except:
      print('No HREF')
      href.append('NONE')
    try:
        response = requests.get(
            'https://en.wikipedia.org/w/api.php',
            params={
                'action': 'query',
                'format': 'json',
                'titles': litag.a.get('title'),
                'prop': 'extracts',
                'explaintext': True,
                'exlimit': 'max',
            }
        ).json()
        raw.append(response)
    except:
        raw.append('NONE')
    try:
        tempinfluencers, tempinfluenced = bio_table(wikiurl)
        influencers.append(tempinfluencers)
        influenced.append(tempinfluenced)
    except:
        influencers.append([])
        influenced.append([])
    phil.append(litag.text)
    print(litag.text)

Philosophers from R to Z

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_philosophers_(R%E2%80%93Z)"
page = urllib.request.urlopen(url)
soup = bs(page, "lxml")

In [0]:
for ultag in soup.find_all('ul'):
  for litag in ultag.find_all('li'):
    try:
      print('HREF: ' + litag.a.get('href'))
      wikiurl = starturl + litag.a.get('href')
      print(wikiurl)
      href.append(wikiurl)
    except:
      print('No HREF')
      href.append('NONE')
    try:
        response = requests.get(
            'https://en.wikipedia.org/w/api.php',
            params={
                'action': 'query',
                'format': 'json',
                'titles': litag.a.get('title'),
                'prop': 'extracts',
                'explaintext': True,
                'exlimit': 'max',
            }
        ).json()
        raw.append(response)
    except:
        raw.append('NONE')
    try:
        tempinfluencers, tempinfluenced = bio_table(wikiurl)
        influencers.append(tempinfluencers)
        influenced.append(tempinfluenced)
    except:
        influencers.append([])
        influenced.append([])
    phil.append(litag.text)
    print(litag.text)

In [0]:
df = pd.DataFrame(phil,columns=['Philosopher'])
df['href']=href
df['raw']=raw
df['influencers']=influencers
df['influenced']=influenced
df
df['raw'].sample(n=3)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df)

In [0]:
result = df.to_json(orient='index')
from google.colab import files
open('result.json', 'w').write(result)
files.download('result.json')

Transform the result

In [0]:
with open('result.json', 'r') as f:
  contents = json.loads(f.read())
print(contents['67']['raw']['query']['pages'])
print(contents['67']['raw'])
json_list = []
for content in contents:
  json_object = {}
  json_object['table'] = {}
  try:
    json_object['pageid'] = list(contents[content]['raw']['query']['pages'].keys())[0]
    json_object['philosopher'] = contents[content]['raw']['query']['pages'][json_object["pageid"]]['title']
    json_object['article'] = contents[content]['raw']['query']['pages'][json_object['pageid']]['extract']
    json_object['table']['influencers'] = contents[content]['influencers']
    json_object['table']['influenced'] = contents[content]['influenced']
    json_list.append(json_object)
  except:
    print('No page, scajella')
print(json_list[67])
with open('final_result.json', 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)
files.download('final_result.json')